In [2]:
import json
import pickle
from collections import defaultdict

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report

from configs.globals import DEVICE
from data.create_fold import SuperInputData


class WindowClassifierWithTransformer(nn.Module):
    def __init__(self, input_dim=100, hidden_dim=64, num_classes=7,
                 num_layers=2, nhead=8, dropout=0.1, num_windows=9):
        super(WindowClassifierWithTransformer, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)

        self.positional_encoding = nn.Parameter(torch.randn(1, num_windows, hidden_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Linear(hidden_dim, num_classes)

        self.reduction_layer = nn.Linear(num_windows * num_classes, num_classes)


    def forward(self, x):
        x = self.embedding(x)

        x = x + self.positional_encoding

        x = x.transpose(0, 1)

        x = self.transformer_encoder(x)

        x = x.transpose(0, 1)

        logits = self.classifier(x)

        flattened = logits.reshape(logits.size(0), logits.size(1) * logits.size(2))

        final_logits = self.reduction_layer(flattened)  # [batch_size, num_classes]

        final_probs = F.softmax(final_logits, dim=-1)  # [batch_size, num_classes]

        return final_probs


# Example usage:
if __name__ == "__main__":
    # Create a sample input: batch_size=2, num_windows=9, input_dim=3
    sample_input = torch.randn(2, 9, 100)

    # Instantiate the classifier
    model = WindowClassifierWithTransformer()

    # Forward pass through the model
    output = model(sample_input)
    print(output.shape)  # Expected shape: (2, 9, 7)

torch.Size([2, 7])


/Users/vitor/Desktop/mestrado/ingred/.venv/lib/python3.9/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [3]:
class CategoryClassifier(nn.Module):
    def __init__(self, input_dim=100, hidden_dim=64, num_classes=7):
        super(CategoryClassifier, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dropout1 = nn.Dropout(p=0.5)

        self.fc_inter  = nn.Linear(hidden_dim, 32)
        self.bn2       = nn.BatchNorm1d(32)
        self.dropout2  = nn.Dropout(p=0.5)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        x = self.fc_inter(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc(x)
        return x

In [4]:
x = torch.randn(32, 9, 100)  # Example input

model = WindowClassifierWithTransformer()
output = model(x)

In [5]:
from tqdm.notebook import tqdm
import pickle

# Load a pickle file
fold_results = None

In [6]:
with open('/Users/vitor/Desktop/mestrado/ingred/data/output/florida/pre-processing/folds.pkl', 'rb') as f:
    fold_results = pickle.load(f)
metrics_history = {}

UnpicklingError: unpickling stack underflow

In [7]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold

# --- your pre‐processing steps ---
df = pd.read_csv('/Users/vitor/Desktop/mestrado/ingred/data/ori/pre-processing/georgia/categorypoi-input.csv')
df['y'] = df['category'].astype('category').cat.codes
df.drop(columns=['placeid', 'category'], inplace=True)
feature_cols = df.columns[0:100]
df['x'] = df[feature_cols].values.tolist()

# --- wrap into a Dataset ---
class POIDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# prepare raw lists
X = df['x'].tolist()
y = df['y'].tolist()

# --- create 5‐fold splits ---
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds=[]

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), 1):
    # slice out this fold’s train/val
    X_train = [X[i] for i in train_idx]
    y_train = [y[i] for i in train_idx]
    X_val   = [X[i] for i in val_idx]
    y_val   = [y[i] for i in val_idx]

    # build Datasets
    train_ds = POIDataset(X_train, y_train)
    val_ds   = POIDataset(X_val,   y_val)

    # and DataLoaders
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=4,persistent_workers=True)
    val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=4,persistent_workers=True)

    folds.append((train_loader, val_loader))

    print(f"Fold {fold}: train={len(train_ds)}  val={len(val_ds)}")

Fold 1: train=18761  val=4691
Fold 2: train=18761  val=4691
Fold 3: train=18762  val=4690
Fold 4: train=18762  val=4690
Fold 5: train=18762  val=4690


In [8]:
for fold_idx, (train_loader, val_loader) in enumerate(folds):
    fold_metrics = defaultdict(list)

    model = CategoryClassifier()
    model.to(DEVICE)


    all_train_labels = []
    for data_next in train_loader:
        y_next = data_next.labels.to(DEVICE, non_blocking=True)
        y_next = y_next.squeeze()
        all_train_labels.extend(y_next.cpu().numpy())
    all_train_labels = np.array(all_train_labels)
    class_counts = np.bincount(all_train_labels)
    class_weights = 1.0 / (class_counts + 1e-6)
    class_weights = class_weights / class_weights.sum()
    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.005,
        epochs=500,
        steps_per_epoch=len(train_loader)
    )

    best_val_acc = 0.0
    epoch_progress = tqdm(range(500), desc=f"Fold {fold_idx}")

    for epoch in epoch_progress:
        model.train()
        train_loss = 0.0
        train_acc = 0.0
        train_total = 0

        for data_next in train_loader:
            x_next = data_next.features.to(DEVICE, non_blocking=True)
            x_next = x_next.view(x_next.size(0), -1)
            y_next = data_next['y'].to(DEVICE, non_blocking=True)
            y_next = y_next.squeeze()

            optimizer.zero_grad()
            out_a = model(x_next)
            loss = criterion(out_a, y_next)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            if DEVICE.type == 'mps':
                torch.mps.synchronize()

            optimizer.step()
            scheduler.step()

            _, predicted = torch.max(out_a, 1)
            correct = (predicted == y_next).sum().item()
            total = y_next.size(0)

            train_loss += loss.item() * total
            train_acc += correct
            train_total += total

        epoch_train_loss = train_loss / train_total
        epoch_train_acc = train_acc / train_total

        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        val_total = 0

        with torch.no_grad():
            for data_next in val_loader:
                x_next = data_next.features.to(DEVICE, non_blocking=True)
                x_next = x_next.view(x_next.size(0), -1)
                y_next = data_next.labels.to(DEVICE, non_blocking=True)
                y_next = y_next.squeeze()

                out_a = model(x_next)
                loss = criterion(out_a, y_next)

                _, predicted = torch.max(out_a, 1)
                correct = (predicted == y_next).sum().item()
                total = y_next.size(0)

                val_loss += loss.item() * total
                val_acc += correct
                val_total += total

        epoch_val_loss = val_loss / val_total
        epoch_val_acc = val_acc / val_total

        fold_metrics['train_loss'].append(epoch_train_loss)
        fold_metrics['train_acc'].append(epoch_train_acc)
        fold_metrics['val_loss'].append(epoch_val_loss)
        fold_metrics['val_acc'].append(epoch_val_acc)

        epoch_progress.set_postfix({
            'tr_loss': f"{epoch_train_loss:.4f}",
            'tr_acc': f"{epoch_train_acc:.4f}",
            'vl_loss': f"{epoch_val_loss:.4f}",
            'vl_acc': f"{epoch_val_acc:.4f}"
        })


    metrics_history[i_fold] = dict(fold_metrics)

    model.eval()

    with torch.no_grad():
        predicted = []
        ground_truth = []
        for data_next in val_loader:
            x_next = data_next.features.to(DEVICE, non_blocking=True)
            x_next = x_next.view(x_next.size(0), -1)
            y_next = data_next.labels.to(DEVICE, non_blocking=True)
            y_next = y_next.squeeze()

            out_a = model(x_next)

            _, pred = torch.max(out_a, 1)
            predicted.append(pred.cpu().numpy())
            ground_truth.append(y_next.cpu().numpy())

        report = classification_report(
            np.concatenate(ground_truth),
            np.concatenate(predicted),
            output_dict=True,
            zero_division=0
        )
        print(json.dumps(report, indent=4))

    print(f"Fold {fold_idx} - Best Val Acc: {best_val_acc:.4f}")

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'POIDataset' on <module '__main__' (built-in)>


KeyboardInterrupt: 

In [ ]:
for fold_idx, (i_fold, dataloader) in enumerate(fold_results.items()):
    dataloader_next = dataloader['next']
    fold_metrics = defaultdict(list)

    model = WindowClassifierWithTransformer()
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.01,
        epochs=200,
        steps_per_epoch=len(dataloader_next.train.dataloader)
    )

    best_val_acc = 0.0
    epoch_progress = tqdm(range(200), desc=f"Fold {fold_idx}")

    for epoch in epoch_progress:
        model.train()
        train_loss = 0.0
        train_acc = 0.0
        train_total = 0

        for data_next in dataloader_next.train.dataloader:
            x_next = data_next['x'].to(DEVICE, non_blocking=True)
            y_next = data_next['y'].to(DEVICE, non_blocking=True)

            optimizer.zero_grad()
            out_a = model(x_next)
            loss = criterion(out_a, y_next)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            if DEVICE.type == 'mps':
                torch.mps.synchronize()

            optimizer.step()
            scheduler.step()

            _, predicted = torch.max(out_a, 1)
            correct = (predicted == y_next).sum().item()
            total = y_next.size(0)

            train_loss += loss.item() * total
            train_acc += correct
            train_total += total

        epoch_train_loss = train_loss / train_total
        epoch_train_acc = train_acc / train_total

        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        val_total = 0

        with torch.no_grad():
            for data_next in dataloader_next.val.dataloader:
                x_next = data_next['x'].to(DEVICE, non_blocking=True)
                y_next = data_next['y'].to(DEVICE, non_blocking=True)

                out_a = model(x_next)
                loss = criterion(out_a, y_next)

                _, predicted = torch.max(out_a, 1)
                correct = (predicted == y_next).sum().item()
                total = y_next.size(0)

                val_loss += loss.item() * total
                val_acc += correct
                val_total += total

        epoch_val_loss = val_loss / val_total
        epoch_val_acc = val_acc / val_total

        fold_metrics['train_loss'].append(epoch_train_loss)
        fold_metrics['train_acc'].append(epoch_train_acc)
        fold_metrics['val_loss'].append(epoch_val_loss)
        fold_metrics['val_acc'].append(epoch_val_acc)

        epoch_progress.set_postfix({
            'tr_loss': f"{epoch_train_loss:.4f}",
            'tr_acc': f"{epoch_train_acc:.4f}",
            'vl_loss': f"{epoch_val_loss:.4f}",
            'vl_acc': f"{epoch_val_acc:.4f}"
        })


    metrics_history[i_fold] = dict(fold_metrics)

    model.eval()

    with torch.no_grad():
        predicted = []
        ground_truth = []
        for data_next in dataloader_next.val.dataloader:
            x_next = data_next['x'].to(DEVICE, non_blocking=True)
            y_next = data_next['y'].to(DEVICE, non_blocking=True)

            out_a = model(x_next)

            _, pred = torch.max(out_a, 1)
            predicted.append(pred.cpu().numpy())
            ground_truth.append(y_next.cpu().numpy())

        report = classification_report(
            np.concatenate(ground_truth),
            np.concatenate(predicted),
            output_dict=True,
            zero_division=0
        )
        print(json.dumps(report, indent=4))

    print(f"Fold {fold_idx} - Best Val Acc: {best_val_acc:.4f}")

In [ ]:
output.shape

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Cross-Stitch Unit
class CrossStitchUnit(nn.Module):
    def __init__(self):
        super(CrossStitchUnit, self).__init__()
        # Learnable alpha parameters (initialized to identity)
        self.alpha = nn.Parameter(torch.tensor([[0.9, 0.1], [0.1, 0.9]], requires_grad=True))

    def forward(self, a, b):
        # a and b are features from each task branch
        a_out = self.alpha[0, 0] * a + self.alpha[0, 1] * b
        b_out = self.alpha[1, 0] * a + self.alpha[1, 1] * b
        return a_out, b_out


# A simple convolutional block
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        return self.pool(F.relu(self.conv(x)))


# Main Multi-Task Network with Cross-Stitch
class CrossStitchNet(nn.Module):
    def __init__(self):
        super(CrossStitchNet, self).__init__()
        # Task-specific initial blocks
        self.taskA_conv1 = ConvBlock()
        self.taskB_conv1 = ConvBlock()

        # Cross-stitch unit after first conv layer
        self.cross_stitch = CrossStitchUnit()

        # Shared second conv block
        self.taskA_conv2 = ConvBlock()
        self.taskB_conv2 = ConvBlock()

        # Task-specific heads
        self.taskA_fc = nn.Linear(16 * 7 * 7, 10)  # For classification
        self.taskB_fc = nn.Linear(16 * 7 * 7, 1)  # For regression

    def forward(self, x):
        a = self.taskA_conv1(x)
        b = self.taskB_conv1(x)

        # Cross-stitch blending
        a, b = self.cross_stitch(a, b)

        # Continue task-specific paths
        a = self.taskA_conv2(a)
        b = self.taskB_conv2(b)

        # Flatten
        a = a.view(a.size(0), -1)
        b = b.view(b.size(0), -1)

        # Final heads
        outA = self.taskA_fc(a)
        outB = self.taskB_fc(b)
        return outA, outB

In [ ]:
x = torch.randn(32, 1, 28, 28)  # Example input

model = CrossStitchNet()
output = model(x)

In [ ]:
output.shape